In [9]:
def recommend (user_id):
    import pandas as pd

    #csv파일 dataframe으로 변환하고 읽기
    review_data_2 = pd.read_csv('./yogiyo/order_db_v2.csv', encoding='ansi')
                              #encoding='utf-8')
    #불필요한 데이터 drop하기
    review_data_2.drop('Unnamed: 0', axis=1, inplace=True)

    menu_data = pd.read_csv('./yogiyo/Menu_DB.csv', encoding='ansi')
                          #encoding='utf-8')
    #menu_data, review_data_2 merge 시키기
    merge_data = pd.merge(review_data_2, menu_data, left_on='order_item', 
                          right_on='menu_name', how='outer')
    merge_data.drop(['menu_name','menu_exp'], axis=1, inplace=True)
    
    #'comment'를 제외한 모든 변수들 붙인다
    cols_to_move = ['comment']
    cols = [col for col in merge_data if col not in cols_to_move] + cols_to_move
    
    group_data = merge_data[cols].groupby(['menu_categori'])
    
    main_menu_group = group_data.get_group('메인')
    side_menu_group = group_data.get_group('사이드')
    
    user_menu_rating = main_menu_group.pivot_table(
        'point', 
        index = 'user_id', 
        columns = 'order_item').fillna(0)
    
    import numpy as np

    #matrix는 위에서만든 pivot_table값을 numpy matrix로 만듬
    matrix = user_menu_rating.values

    #user_rating_avg는 사용자의 평균 맛평가
    user_rating_avg = np.mean(matrix, axis = 1)

    #matrix_user_avg : 사용자-평가에대해 사용자 평균 평점을 뺀것
    matrix_user_avg = matrix - user_rating_avg.reshape(-1, 1)
    
    from scipy.sparse.linalg import svds
    #scipy에서제공하는 svd
    #U행렬, sigma행렬, V전치행렬 반환
    U, sigma, Vt = svds(matrix_user_avg, k=2)
    
    sigma = np.diag(sigma)
    
    #matrix_user_avg를 svd를 적용해 분해한 상태이므로 원복 필요
    svd_user_predicted_rating = np.dot(np.dot(U, sigma), Vt) + user_rating_avg.reshape(-1,1)
    
    svd_preds = pd.DataFrame(svd_user_predicted_rating, 
                             index=user_menu_rating.index, 
                             columns=user_menu_rating.columns)
    
    def recommend_menu(svd_preds, og_user_id, og_order_item, og_point, num_recommendations=5):
    
        #맛평가 평점이 높은 순으로 정렬
        sort_user_predictions = svd_preds.index.sort_values(ascending=False)

        #원본데이터에서 user_id에 해당하는 데이터 뽑기
        user_data = og_point[og_point.user_id == og_user_id]

        #원본데이터에서 사용자가 주문했던 음식은 제외한 데이터 추출
        recommendations = og_order_item[-og_order_item['order_item'].isin(user_data['order_item'])]  
        #사용자의 메뉴 평점이 높은 순으로 정렬된 데이터와 recommendations를 합친다
        recommendations = recommendations.merge(pd.DataFrame(sort_user_predictions).reset_index(), on = 'user_id')
        #컬럼이름을 바꾸고 정렬해서 return
        recommendations = recommendations.rename(columns = {'index' : 'Predictions'}).sort_values('Predictions', ascending=False)

        return user_data, recommendations
    
    already_rated_main, predictions_main = recommend_menu(svd_preds, user_id, main_menu_group, 
                                            main_menu_group, 10)
    already_rated_main = already_rated_main.sort_values(by='order_no', ascending=False)
    #predictions = predictions.sort_values(by='order_no', ascending=False)

    already_rated_side, predictions_side = recommend_menu(svd_preds, user_id, side_menu_group, 
                                                side_menu_group, 10)
    already_rated_side = already_rated_side.sort_values(by='order_no', ascending=False)
    
    return already_rated_main, predictions_main.head(10), already_rated_side, predictions_side.head(10)

In [10]:
recommend('dk')

(      user_id  order_date         order_item  order_no  point menu_categori  \
 1635       dk  2020-11-23        두툼한 통등심 가츠동      9178      5            메인   
 1644       dk  2020-11-22        두툼한 통등심 가츠동      9130      5            메인   
 5990       dk  2020-10-27             대패삼겹덮밥      8866      5            메인   
 11179      dk  2020-10-26       그라나파다노 김치볶음밥      8861      5            메인   
 11192      dk  2020-10-10       그라나파다노 김치볶음밥      8687      5            메인   
 3464       dk  2020-09-07  로스트 치킨과 매콤한 크림리조또      8304      5            메인   
 1771       dk  2020-09-07        두툼한 통등심 가츠동      8303      4            메인   
 1832       dk  2020-08-13        두툼한 통등심 가츠동      7949      1            메인   
 14466      dk  2020-03-24        어향가지 덮밥&멘보샤      6622      5            메인   
 10126      dk  2020-03-24            어향가지 덮밥      6622      5            메인   
 12910      dk  2020-03-16           수원왕갈비 치밥      6503      5            메인   
 417        dk  2020-03-14           궁중식